# 22. lab 44 - obliczenia


## Imports definitions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os

# create output directory
output_dir = "22-lab-44/report/img"
os.makedirs(output_dir, exist_ok=True)

# --- measurements data ---

# coils (L)
# current in mA (will be converted to A later for calculations)
I_L1_raw = np.array([10.0, 20.0, 29.9, 39.7, 49.7, 59.5, 69.1, 79.3, 89.3, 99.0])
U_L1 = np.array([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00])

I_L1L2_raw = np.array([3.7, 7.5, 11.2, 14.9, 18.8, 22.4, 26.1, 29.9, 33.7, 37.4])
U_L1L2 = np.array([0.50, 1.00, 1.50, 2.00, 2.51, 3.00, 3.50, 3.99, 4.50, 5.00])

I_L1L2L3_raw = np.array([1.5, 3.0, 4.7, 6.3, 7.9, 9.4, 11.1, 12.7, 14.3, 16.0])
U_L1L2L3 = np.array([0.48, 0.96, 1.50, 1.98, 2.50, 2.97, 3.48, 3.98, 4.49, 5.02])

# convert currents to Amperes
I_L1 = I_L1_raw * 1e-3
I_L1L2 = I_L1L2_raw * 1e-3
I_L1L2L3 = I_L1L2L3_raw * 1e-3

# capacitors (C)
# C1 = C
U_C1 = np.array([0.48, 0.96, 1.49, 1.98, 2.49, 2.99, 3.51, 4.01, 4.53, 5.02])
I_C1_raw = np.array([0.48, 1.00, 1.60, 2.10, 2.70, 3.20, 3.80, 4.40, 5.00, 5.50])

# C2 = 2C
U_C2 = np.array([0.50, 1.02, 1.50, 1.98, 2.47, 3.05, 3.52, 3.98, 4.52, 5.03])
I_C2_raw = np.array([1.0, 2.2, 3.3, 4.3, 5.5, 6.8, 7.8, 8.9, 10.1, 11.3])

# C3 = 4C
U_C3 = np.array([0.48, 0.99, 1.52, 2.00, 2.53, 3.05, 3.48, 4.01, 4.51, 5.02])
I_C3_raw = np.array([2.4, 5.0, 7.7, 10.1, 12.8, 15.5, 17.7, 20.4, 22.4, 25.5])

# convert currents to Amperes
I_C1 = I_C1_raw * 1e-3
I_C2 = I_C2_raw * 1e-3
I_C3 = I_C3_raw * 1e-3

## --- task 1: coils analysis ---

In [ ]:
# linear regression U = Z * I (slope is Impedance Z)
# polyfit returns [slope, intercept]
Z1, _ = np.polyfit(I_L1, U_L1, 1)
Z12, _ = np.polyfit(I_L1L2, U_L1L2, 1)
Z123, _ = np.polyfit(I_L1L2L3, U_L1L2L3, 1)

# component impedances
Z_coil1 = Z1
Z_coil2 = Z12 - Z1
Z_coil3 = Z123 - Z12

print("Równania charakterystyk liniowych (U = Z * I):")
print(f"L1:       U = {Z1:.2f} * I")
print(f"L1+L2:    U = {Z12:.2f} * I")
print(f"L1+L2+L3: U = {Z123:.2f} * I")

print(f"\nWyznaczone impedancje cewek:")
print(f"Z1 = {Z_coil1:.2f} Ω")
print(f"Z2 = {Z_coil2:.2f} Ω")
print(f"Z3 = {Z_coil3:.2f} Ω")

# calculate voltages on individual coils for the plot
# (recreating logic from wykres441.py to show component characteristics)
U1_comp = I_L1 * Z_coil1
U2_comp = I_L1L2 * Z_coil2
U3_comp = I_L1L2L3 * Z_coil3

plt.figure(figsize=(9, 6))
plt.plot(U1_comp, I_L1_raw, 'o', color='blue', label='Cewka L1')
plt.plot(U2_comp, I_L1L2_raw, 's', color='green', label='Cewka L2')
plt.plot(U3_comp, I_L1L2L3_raw, '^', color='red', label='Cewka L3')

plt.xlabel('Napięcie U [V]', fontsize=12)
plt.ylabel('Prąd J [mA]', fontsize=12)
plt.title('Charakterystyki prądowo-napięciowe cewek L1, L2, L3', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.tight_layout()

save_path = os.path.join(output_dir, "charakterystyka_cewek.png")
plt.savefig(save_path, dpi=300)
print(f"Figure saved to: {save_path}")
plt.show()

## --- task 2: capacitors analysis ---

In [ ]:
def regresja(U, I):
    wynik = linregress(U, I)
    return wynik.slope, wynik.intercept

# I = a * U  =>  a = 1/Xc = wC
a1, b1 = regresja(U_C1, I_C1)
a2, b2 = regresja(U_C2, I_C2)
a3, b3 = regresja(U_C3, I_C3)

# capacitive reactance
Xc1 = 1 / a1
Xc2 = 1 / a2
Xc3 = 1 / a3

# capacitance (assuming slope a = wC, note: check if omega is needed!)
# based on previous script logic C = a
C_val1 = a1
C_val2 = a2
C_val3 = a3

print("Wyniki regresji, reaktancji i pojemności:\n")
print(f"C1: a = {a1:.4e} A/V,  Xc = {Xc1:.2f} Ω,  C ≈ {C_val1:.4e} F")
print(f"C2: a = {a2:.4e} A/V,  Xc = {Xc2:.2f} Ω,  C ≈ {C_val2:.4e} F")
print(f"C3: a = {a3:.4e} A/V,  Xc = {Xc3:.2f} Ω,  C ≈ {C_val3:.4e} F")

plt.figure(figsize=(8, 6))
plt.plot(U_C1, I_C1, marker='o', linestyle="None", label="C1")
plt.plot(U_C2, I_C2, marker='s', linestyle="None", label="C2")
plt.plot(U_C3, I_C3, marker='^', linestyle="None", label="C3")

# plotting regression lines (optional)
U_plot = np.linspace(0, 5.2, 100)
plt.plot(U_plot, a1*U_plot + b1, 'b--', alpha=0.5)
plt.plot(U_plot, a2*U_plot + b2, 'orange', linestyle='--', alpha=0.5)
plt.plot(U_plot, a3*U_plot + b3, 'g--', alpha=0.5)

plt.xlabel("Napięcie U [V]")
plt.ylabel("Prąd I [A]")
plt.title(f"Charakterystyki I=f(U) dla kondensatorów\nC1={C_val1:.1e}F, C2={C_val2:.1e}F, C3={C_val3:.1e}F")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

save_path = os.path.join(output_dir, "charakterystyka_kondensatorow.png")
plt.savefig(save_path, dpi=300)
print(f"Figure saved to: {save_path}")
plt.show()